In [65]:
from pdbfixer import PDBFixer

# Load input structure from PDB file into pdbfixer
fixer = PDBFixer('data/2x6m.pdb')

# Inspect topology
print(fixer.topology)

<Topology; 4 chains, 340 residues, 1201 atoms, 1016 bonds>


In [66]:
from collections import defaultdict

for chain in fixer.topology.chains():
    print(f' ------------------- \n Chain ID: {chain.id}')
    chain_residues = defaultdict(int)
    for residue in chain.residues():
        chain_residues[residue.name] += 1
    for residue_name, count in chain_residues.items():
        print(f'  {residue_name}: {count} residues')

 ------------------- 
 Chain ID: A
  GLY: 19 residues
  GLN: 6 residues
  LEU: 6 residues
  VAL: 8 residues
  GLU: 5 residues
  SER: 16 residues
  ALA: 10 residues
  ARG: 7 residues
  CYS: 4 residues
  ILE: 4 residues
  ASP: 5 residues
  TYR: 7 residues
  MET: 2 residues
  TRP: 3 residues
  PHE: 4 residues
  PRO: 3 residues
  LYS: 5 residues
  ASN: 5 residues
  THR: 6 residues
  HIS: 1 residues
 ------------------- 
 Chain ID: B
  ASP: 1 residues
  TYR: 1 residues
  GLU: 2 residues
  PRO: 1 residues
  ALA: 1 residues
 ------------------- 
 Chain ID: A
  HOH: 194 residues
 ------------------- 
 Chain ID: B
  HOH: 14 residues


In [67]:
# Let's remove waters from the structure
fixer.removeHeterogens(keepWater=False)
print(fixer.topology)

<Topology; 2 chains, 132 residues, 993 atoms, 1016 bonds>


In [68]:
# Let's repeat the previous cell
for chain in fixer.topology.chains():
    print(f' ------------------- \n Chain ID: {chain.id}')
    chain_residues = defaultdict(int)
    for residue in chain.residues():
        chain_residues[residue.name] += 1
    for residue_name, count in chain_residues.items():
        print(f'  {residue_name}: {count} residues')

 ------------------- 
 Chain ID: A
  GLY: 19 residues
  GLN: 6 residues
  LEU: 6 residues
  VAL: 8 residues
  GLU: 5 residues
  SER: 16 residues
  ALA: 10 residues
  ARG: 7 residues
  CYS: 4 residues
  ILE: 4 residues
  ASP: 5 residues
  TYR: 7 residues
  MET: 2 residues
  TRP: 3 residues
  PHE: 4 residues
  PRO: 3 residues
  LYS: 5 residues
  ASN: 5 residues
  THR: 6 residues
  HIS: 1 residues
 ------------------- 
 Chain ID: B
  ASP: 1 residues
  TYR: 1 residues
  GLU: 2 residues
  PRO: 1 residues
  ALA: 1 residues


The main purpose of the PDBFixer, however, is to fix the structure:
- Add missing residues
- Add missing heavy atoms (for example on terminal residues or on broken residues)
- Add hydrogens (could also be done with Modeller - other OpenMM tool)

In [69]:
# So let's try to fix this structure
fixer.findMissingResidues()
fixer.findMissingAtoms()
print('Missing residues are:\n', fixer.missingResidues)
print('Missing atoms are:\n', fixer.missingAtoms)
print('Missing terminals are:\n', fixer.missingTerminals)

Missing residues are:
 {}
Missing atoms are:
 {}
Missing terminals are:
 {<Residue 125 (HIS) of chain 0>: ['OXT']}


How does it know about the missing atoms and missing residues?

Easy, for residues - just check out the SEQRES section of your PDB file :)

For atoms, PDBFixer knows the templates of standard amino acids, so it knows when some heavy atom is missing. 
By the way, the templates also give PDBFixer the information about bonds between the atoms. You can see that 
there is no CONNECT records in the PDB file, but the topology contains all the bonds.

For terminals, PDBFixer checks whether the last residue of the chain has the OXT atom, if not - it will add it

In [70]:
# Now let's add these atoms

#For visibility, let's print the topology before and after the addition of atoms to compare
print('Topology before fixing: ', fixer.topology)
fixer.addMissingAtoms()
print('Topology after fixing: ', fixer.topology)

Topology before fixing:  <Topology; 2 chains, 132 residues, 993 atoms, 1016 bonds>
Topology after fixing:  <Topology; 2 chains, 132 residues, 994 atoms, 1017 bonds>


What regarding the Hydrogen addition? I personally prefer the Modeller hydrogen addition over the PDBFixer one, as it provides more control over the process. You can compare the functions yourself by reading their descriptions:


Modeller's addHydrogens():
    Add missing hydrogens to the model.

    Some residues can exist in multiple forms depending on the pH and properties of the local environment. These variants differ in the presence or absence of particular hydrogens. In particular, the following variants are supported:

    Aspartic acid:

    ASH: Neutral form with a hydrogen on one of the delta oxygens
    ASP: Negatively charged form without a hydrogen on either delta oxygen

    Cysteine:

    CYS: Neutral form with a hydrogen on the sulfur
    CYX: No hydrogen on the sulfur (either negatively charged, or part of a disulfide bond)

    Glutamic acid:

    GLH: Neutral form with a hydrogen on one of the epsilon oxygens
    GLU: Negatively charged form without a hydrogen on either epsilon oxygen

    Histidine:

    HID: Neutral form with a hydrogen on the ND1 atom
    HIE: Neutral form with a hydrogen on the NE2 atom
    HIP: Positively charged form with hydrogens on both ND1 and NE2
    HIN: Negatively charged form without a hydrogen on either ND1 or NE2

    Lysine:

    LYN: Neutral form with two hydrogens on the zeta nitrogen
    LYS: Positively charged form with three hydrogens on the zeta nitrogen

    The variant to use for each residue is determined by the following rules:

    The most common variant at the specified pH is selected.
    Any Cysteine that participates in a disulfide bond uses the CYX variant regardless of pH.
    For a neutral Histidine residue, the HID or HIE variant is selected based on which one forms a better hydrogen bond.
    You can override these rules by explicitly specifying a variant for any residue. To do that, provide a list for the 'variants' parameter, and set the corresponding element to the name of the variant to use.

    A special case is when the model already contains a hydrogen that should not be present in the desired variant. If you explicitly specify a variant using the 'variants' parameter, the residue will be modified to match the desired variant, removing hydrogens if necessary. On the other hand, for residues whose variant is selected automatically, this function will only add hydrogens. It will never remove ones that are already present in the model, regardless of the specified pH.

    In all cases, the positions of existing atoms (including existing hydrogens) are not modified.

    Definitions for standard amino acids and nucleotides are built in. You can call loadHydrogenDefinitions() to load additional definitions for other residue types.
    
    Parameters
        forcefield : ForceField=None
        the ForceField to use for determining the positions of hydrogens. If this is None, positions will be picked which are generally reasonable but not optimized for any particular ForceField.

        pH : float=7.0
        the pH based on which to select variants

        variants : list=None
        an optional list of variants to use. If this is specified, its length must equal the number of residues in the model. variants[i] is the name of the variant to use for residue i (indexed starting at 0). If an element is None, the standard rules will be followed to select a variant for that residue. Alternatively, an element may specify exactly which hydrogens to add. In that case, variants[i] should be a list of tuples [(name1, parent1), (name2, parent2), ...]. Each tuple specifies the name of a hydrogen and the name of the parent atom it should be bonded to.

        platform : Platform=None
        the Platform to use when computing the hydrogen atom positions. If this is None, the default Platform will be used.

        residueTemplates : dict=dict()
        specifies which template the ForceField should use for particular residues. The keys should be Residue objects from the Topology, and the values should be the names of the templates to use for them. This is useful when a ForceField contains multiple templates that can match the same residue (e.g Fe2+ and Fe3+ templates in the ForceField for a monoatomic iron ion in the Topology).

PDBFixer's addMissingHydrogens():
    Add missing hydrogen atoms to the structure.

    Parameters
    pH : float, optional, default=7.0
    The pH based on which to select hydrogens.

    forcefield : ForceField, optional, default=None
    The forcefield used when adding and minimizing hydrogens. If None, a default forcefield is used.

    Notes
    No extensive electrostatic analysis is performed; only default residue pKas are used. The pH is only taken into account for standard amino acids.



In [71]:
from openmm.app import Modeller, ForceField

modeller = Modeller(fixer.topology, fixer.positions)
print('Modeller topology: ', modeller.topology)
print('PDBFixer topology: ', fixer.topology)

Modeller topology:  <Topology; 2 chains, 132 residues, 994 atoms, 1017 bonds>
PDBFixer topology:  <Topology; 2 chains, 132 residues, 994 atoms, 1017 bonds>


In [72]:
# To add hydrogens we need to initialize forcefield parameters. We will come back to forcefield details
# later, so for now let's just do it

ff = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [73]:
# Now let's actually add hydrogens, and see what residues Modeller changed
variants = modeller.addHydrogens(forcefield=ff)
for i, res in enumerate(variants):
    if res:
        print(f'Residue {i}: {res}')
print(f"New Modeller topology: {modeller.topology}")

Residue 20: CYX
Residue 31: CYX
Residue 95: CYX
Residue 104: CYX
Residue 125: HID
New Modeller topology: <Topology; 2 chains, 132 residues, 1921 atoms, 1944 bonds>


In [74]:
# Great, now we finished preparation of input structure - let's save it

from openmm.app import PDBFile

PDBFile.writeFile(modeller.topology, modeller.positions, file=open('data/prep_complex.pdb', 'wt'), keepIds=True)

In [75]:
# Using PDBFile you can both write and read the files. You don't need to use PDBFixer each time

loaded_structure = PDBFile('data/prep_complex.pdb')
print(f"Saved structure topology: {modeller.topology}")
print(f"Loaded strucutre topology: {loaded_structure.topology}")


Saved structure topology: <Topology; 2 chains, 132 residues, 1921 atoms, 1944 bonds>
Loaded strucutre topology: <Topology; 2 chains, 132 residues, 1921 atoms, 1944 bonds>


What about mutations? We can do them with PDBFixer!

It is done with applyMutations() function of PDBFixer object.

For this, you need to provide the mutations list and the chain id of the chain, where the mutations should be applied

the mutation list should contain the mutation in the form of strings.

Each string must include the resName (original), id, and resName (target). For example, ALA-133-GLY will mutate alanine 133 to glycine.

In [79]:
#Let's first print the residues of our peptide

loaded_structure_pdbfixer = PDBFixer('data/prep_complex.pdb')

peptide_chain_id = 'B'

for residue in loaded_structure_pdbfixer.topology.residues():
    if residue.chain.id == peptide_chain_id:
        print(f"Peptide residue {residue.name} has original ID (from PDB) {residue.id}")

Peptide residue ASP has original ID (from PDB) 135
Peptide residue TYR has original ID (from PDB) 136
Peptide residue GLU has original ID (from PDB) 137
Peptide residue PRO has original ID (from PDB) 138
Peptide residue GLU has original ID (from PDB) 139
Peptide residue ALA has original ID (from PDB) 140


Let's inspect the structure. For this, let's use the Protein Viewer extension in VSCode. Navigate to the VSCode Extensions, Install the Protein Viewer. Then right click on the data/prep_complex.pdb file and open it with Protein Viewer

You can see that the AA closest to the binding pocket of the nanobody is ALA 140

Let's mutate the next 3 AAs to alanine

In [81]:
mutations = ['GLU-139-ALA',
             'PRO-138-ALA',
             'GLU-137-ALA']

print(f"the original topology: {loaded_structure_pdbfixer.topology}")

loaded_structure_pdbfixer.applyMutations(mutations=mutations, chain_id='B')

for residue in loaded_structure_pdbfixer.topology.residues():
    if residue.chain.id == peptide_chain_id:
        print(f"Peptide residue {residue.name} has original ID (from PDB) {residue.id}")

print(f"the mutated topology: {loaded_structure_pdbfixer.topology}")

the original topology: <Topology; 2 chains, 132 residues, 1921 atoms, 1944 bonds>
Peptide residue ASP has original ID (from PDB) 135
Peptide residue TYR has original ID (from PDB) 136
Peptide residue ALA has original ID (from PDB) 137
Peptide residue ALA has original ID (from PDB) 138
Peptide residue ALA has original ID (from PDB) 139
Peptide residue ALA has original ID (from PDB) 140
the mutated topology: <Topology; 2 chains, 132 residues, 1892 atoms, 1914 bonds>


Let's check whether there are any missing atoms

In [82]:
# Let's try to find the missing parts
loaded_structure_pdbfixer.findMissingResidues()
loaded_structure_pdbfixer.findMissingAtoms()
print('Missing residues are:\n', loaded_structure_pdbfixer.missingResidues)
print('Missing atoms are:\n', loaded_structure_pdbfixer.missingAtoms)
print('Missing terminals are:\n', loaded_structure_pdbfixer.missingTerminals)
print('Topology before fixing: ', loaded_structure_pdbfixer.topology)
loaded_structure_pdbfixer.addMissingAtoms()
print('Topology after fixing: ', loaded_structure_pdbfixer.topology)


Missing residues are:
 {}
Missing atoms are:
 {}
Missing terminals are:
 {}
Topology before fixing:  <Topology; 2 chains, 132 residues, 1892 atoms, 1914 bonds>
Topology after fixing:  <Topology; 2 chains, 132 residues, 1892 atoms, 1914 bonds>


As you can see - no missing atoms. But to convert GLU or PRO to ALA, we only need to delete atoms. What if we mutuate to something bigger instead?

In [83]:
loaded_structure_pdbfixer_test = PDBFixer('data/prep_complex.pdb')

mutations = ['GLU-139-PHE']

loaded_structure_pdbfixer_test.applyMutations(mutations=mutations, chain_id='B')

loaded_structure_pdbfixer_test.findMissingResidues()
loaded_structure_pdbfixer_test.findMissingAtoms()
print('Missing residues are:\n', loaded_structure_pdbfixer_test.missingResidues)
print('Missing atoms are:\n', loaded_structure_pdbfixer_test.missingAtoms)
print('Missing terminals are:\n', loaded_structure_pdbfixer_test.missingTerminals)
print('Topology before fixing: ', loaded_structure_pdbfixer_test.topology)
loaded_structure_pdbfixer_test.addMissingAtoms()
print('Topology after fixing: ', loaded_structure_pdbfixer_test.topology)

Missing residues are:
 {}
Missing atoms are:
 {<Residue 130 (PHE) of chain 1>: [<Atom 4 (CD1) of chain 0 residue 0 (PHE)>, <Atom 5 (CE1) of chain 0 residue 0 (PHE)>, <Atom 6 (CZ) of chain 0 residue 0 (PHE)>, <Atom 7 (CE2) of chain 0 residue 0 (PHE)>, <Atom 8 (CD2) of chain 0 residue 0 (PHE)>]}
Missing terminals are:
 {}
Topology before fixing:  <Topology; 2 chains, 132 residues, 1912 atoms, 1935 bonds>
Topology after fixing:  <Topology; 2 chains, 132 residues, 1917 atoms, 1941 bonds>


As you can see - when we mutating to bigger sidechain - we need to addMissingAtoms()

So when mutating amino acids with pdbfixer - remember to call findMissingResidues() --> findMissingAtoms() --> addMissingAtoms()

Let's return to our first mutant structure, where we mutated 3 AAs to alanine. I wonder whether we assigned hydrogens to this structure?

In [96]:
for residue in loaded_structure_pdbfixer.topology.residues():
    if residue.chain.id == 'B' and residue.id in ['137', '138', '139']:
        print(f"Residue {residue.name}{residue.id} contains atoms:")
        for atom in residue.atoms():
            print(f"Atom '{atom.name}', of element: {atom.element.name}")

Residue ALA137 contains atoms:
Atom 'N', of element: nitrogen
Atom 'CA', of element: carbon
Atom 'C', of element: carbon
Atom 'O', of element: oxygen
Atom 'CB', of element: carbon
Residue ALA138 contains atoms:
Atom 'N', of element: nitrogen
Atom 'CA', of element: carbon
Atom 'C', of element: carbon
Atom 'O', of element: oxygen
Atom 'CB', of element: carbon
Residue ALA139 contains atoms:
Atom 'N', of element: nitrogen
Atom 'CA', of element: carbon
Atom 'C', of element: carbon
Atom 'O', of element: oxygen
Atom 'CB', of element: carbon


As you can see - no hydrogens. So let's add them with Modeller as we done previously

In [97]:
from openmm.app import Modeller, ForceField

print(f"Init topology: {loaded_structure_pdbfixer.topology}")

modeller = Modeller(loaded_structure_pdbfixer.topology, loaded_structure_pdbfixer.positions)

ff = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

modeller.addHydrogens(forcefield=ff)

print(f"New Modeller topology: {modeller.topology}")

Init topology: <Topology; 2 chains, 132 residues, 1892 atoms, 1914 bonds>
New Modeller topology: <Topology; 2 chains, 132 residues, 1907 atoms, 1929 bonds>


In [98]:
# Great, now we finished preparation of input structure - let's save it

from openmm.app import PDBFile

PDBFile.writeFile(modeller.topology, modeller.positions, file=open('data/prep_complex_AAA.pdb', 'wt'), keepIds=True)